In [76]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np

In [77]:
cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'u': 'dtrNCBpDaLJaRutT2DQLSih0IUZ5EEkvU0E4PqVsLckLY2l8',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'logged_out_browsing_page_count': '2',
    'locale': 'en',
    'srb_8': '0_wl',
    '_session_id2': 'dd2cbef59506793b2aebe5947688a6fa',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; u=dtrNCBpDaLJaRutT2DQLSih0IUZ5EEkvU0E4PqVsLckLY2l8; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; logged_out_browsing_page_count=2; locale=en; srb_8=0_wl; _session_id2=dd2cbef59506793b2aebe5947688a6fa',
    'If-None-Match': 'W/"959d55aeb336cd1b9148ad7a8b947f48"',
    'Referer': 'https://www.goodreads.com/user/best_reviewers?ref=nav_comm_people',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [47]:
def cleaning_name(raw_name):
    # print(type(result[1]))
    result = re.findall(r"(\w+ ?(\w+)?)", raw_name)
    # print(str(result.group(3)))
    return result[1][0]


In [79]:

response = requests.get('https://www.goodreads.com/user/top_readers', cookies=cookies, headers=headers)
html_soup = BeautifulSoup(response.text, 'html5lib')
reader_table = html_soup.find('table', class_ = "tableList")
reader_data = []

for reader in reader_table.find_all('tr'):
    name = cleaning_name(reader.text)
    link = reader.a['href']
    completeLink = "https://www.goodreads.com" + link
    getGenre = requests.get(completeLink)
    genreParsing = BeautifulSoup(getGenre.text, 'html5lib')
    genreBox = genreParsing.find('div', class_= "stacked clearFloats bigBox")
    if genreBox == None:
        genreList = "Missing"
        continue
    else:
        genreList = []
        x = genreBox.find('div', class_ = "bigBoxBody")
        y = x.find('div', class_ = "bigBoxContent containerWithHeaderContent")
        z = y.find_all('a')
        for i in z:
            genreList.append(i.text)
        print(genreList)

    data = [name,genreList]
    reader_data.append(data)


readers_data = pd.DataFrame(reader_data, columns = ['name','genre'])
readers_data.head()

['Comics', 'Crime', 'Fantasy', 'Fiction', 'Historical fiction', 'Horror', 'Humor and Comedy', 'Manga', 'Music', 'Mystery', 'Romance', 'Science fiction', 'Sports', 'Thriller', 'Young-adult']
['Fantasy', 'Fiction', 'Manga', 'Romance']
['Cookbooks', 'Fiction', 'Graphic novels', 'Historical fiction', 'History', 'Humor and Comedy', 'Mystery', 'Non-fiction', 'Romance', 'Science fiction', 'Young-adult']
['Fantasy', 'Fiction', 'Romance', 'Young-adult']
['', 'October 2022 Most Anticipated Romance Releases']
['Art', 'Biography', 'Classics', 'Comics', 'Crime', 'Ebooks', 'Fantasy', 'Fiction', 'Graphic novels', 'Historical fiction', 'History', 'Horror', 'Humor and Comedy', 'Music', 'Mystery', 'Non-fiction', 'Paranormal', 'Philosophy', 'Psychology', 'Romance', 'Science', 'Science fiction', 'Self help', 'Suspense', 'Thriller', 'Travel', 'Young-adult']
['Fantasy', 'Fiction', 'Mystery', 'Romance', 'Thriller', 'Young-adult']
['Forthcoming Books', '', '', '', '', '', 'Fiksi Kriminal, Detektif, dan Thrill

,name,genre
0,Agra Fawzi,"[Comics, Crime, Fantasy, Fiction, Historical f..."
1,Razellia,"[Fantasy, Fiction, Manga, Romance]"
2,Izzah Gunadumna,"[Cookbooks, Fiction, Graphic novels, Historica..."
3,Dewi Indra,"[Fantasy, Fiction, Romance, Young-adult]"
4,Nicole Potter,"[, October 2022 Most Anticipated Romance Relea..."


In [80]:
readers_data.to_csv('readers.csv', index = False)